In [1]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def = None, max_const_size=32):
    """Visualize TensorFlow graph."""
    
    if graph_def is None:
        graph_def = tf.get_default_graph().as_graph_def()
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat

import os
import logging
import math
import itertools

import scipy.stats
import typing
import random
from time import time
from datetime import datetime

import tensorflow as tf
from sklearn.cross_validation import train_test_split

logging.basicConfig(level=logging.DEBUG)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [3]:
def calculate_account_value(money, stocks, stock_prices: tf.Tensor):
        """
        Calculates the current Marked Value of the account

        Arguments:
        - stock_prices:  tensor[minibatch, company] -> price

        Returns: 
        - tensor[minibatch] -> Current market value
        """
        with tf.name_scope("account_value"):
            return tf.reduce_sum(money, axis=1) + tf.reduce_sum(stocks*stock_prices, axis=1)

class TradingState(typing.NamedTuple):
    stocks:        tf.Tensor   # tensor[minibatch, company] -> number of stocks
    money:         tf.Tensor   # tensor[minibatch] -> [cash available, cash that will be available tomorrow, ...]
    account_value: tf.Tensor   # tensor[minibatch] -> Market value of account
    inner_state:  typing.Any  # Whatever state the trader want to propagate from one state to the next
        
    

In [214]:
FEATURE_LOW    = 0
FEATURE_HIGH   = 1
FEATURE_OPEN   = 2
FEATURE_CLOSE  = 3
FEATURE_VOLUME = 4

RESPONSE_BUY_PRICE   = 0
RESPONSE_BUY_AMOUNT  = 1
RESPONSE_SELL_LOW    = 2
RESPONSE_SELL_HIGH   = 3

MONEY_TODAY = 0
MONEY_TOMORROW = 1
MONEY_AFTER_TOMORROW = 2

SELL_TRANSACTION_COST = 5 # 5 USD per transaction
BUY_TRANSACTION_COST = 5  # 5 USD per transaction

def trader_builder(historic_window_size, trainable_variables_node, first_state_node):    
    with trainable_variables_node:
        coef_buy_price  = tf.Variable(tf.zeros([historic_window_size, 4]), name="buy_coefs")
        coef_sell_price = tf.Variable(tf.zeros([historic_window_size, 4]), name="sell_coefs")
        coef_bias = tf.Variable([-1.,+1.], name="biases")
        
    with first_state_node:
        first_inner_state = tf.constant([1,2,3])
        
    def build_trader(state: TradingState, historic_data: tf.Tensor, inner_state: typing.Any) -> tf.Tensor:
        """
        Builds the trader network on tensorflow

        Arguments:
        - state: Trading state coming from the previous day
        - historic_data: tensor[minibatch, time, company] -> [low, high, open, close]

        Returns: 
        tensor[minibatch, company] -> [buy_price, buy_amount, sell_low_price, sell_high_price]
        - buy_price: The price for which you want to buy stocks
        - buy_amount: Fraction of the cash available to be used to buy stocks from this company (If the price target is reached)
        - sell_high_price: If the stock price reaches more than this amount, all of the owned stocks will be sold (Ideal case, sell when the price is high)
        - sell_low_price: If the stock price reaches less than this amount, all of the owned stocks will also be sold (Fucked up case: Prices are crashing, minimize losses)
        """
        with tf.name_scope('normalize_input'):
            normalize_factor = tf.concat(
                [
                    tf.expand_dims(historic_data[:, 0, :, FEATURE_OPEN ], 1),
                    historic_data[:, :-1, :, FEATURE_CLOSE],
                ],
                axis = 1,
                name = 'factor'
            )
            normalized_history = tf.multiply(
                100., 
                tf.log( historic_data[:,:,:,:-1] / normalize_factor[:, :, :, tf.newaxis]), 
                name='normalized_history'
            )
        
        with tf.name_scope('buy_price'):
            buy_price = coef_bias[0] + tf.reduce_sum(normalized_history * coef_buy_price[tf.newaxis, :, tf.newaxis, :], axis=[1,3])
        with tf.name_scope('buy_amount'):
            buy_amount = tf.fill(tf.shape(buy_price), value=0.1)
        with tf.name_scope('sell_low_price'):
            sell_low_price = tf.fill(tf.shape(buy_price), value=-9999.)  # Never!
        with tf.name_scope('sell_high_price'):
            sell_high_price = coef_bias[1] + tf.reduce_sum(normalized_history * coef_sell_price[tf.newaxis, :, tf.newaxis, :], axis=[1,3])

        with tf.name_scope('denormalize_prices'):
            def denorm(x, name=None):
                return tf.multiply(tf.exp(x / 100), historic_data[:, -1, :, FEATURE_CLOSE], name = name)
            buy_price = denorm(buy_price, 'buy_price')
            sell_low_price = denorm(sell_low_price, 'sell_low_price')
            sell_high_price = denorm(sell_high_price, 'sell_high_price')
            
        with tf.name_scope('result'):
            result = tf.stack([buy_price, buy_amount, sell_low_price, sell_high_price], axis=2)
            return (result, inner_state)
        
    return build_trader, first_inner_state

In [312]:
def build_env_step(build_trader, state: TradingState, historic_data: tf.Tensor, next_day_data: tf.Tensor) -> TradingState:
    """
    Builds a single-day trading environment.
    Arguments:
    - state: Trading state coming from the previous day
    - historic_data: tensor[minibatch, time, company] -> [low, high, open, close]
    - next_day_data: tensor[minibatch, company] -> [low, high, open, close]

    
    Returns: The next state, computed at the end of the next trading day
    """   
    with tf.name_scope("trader"):
        trader, new_inner_state = build_trader(state = state, historic_data = historic_data, inner_state = state.inner_state)

        def smooth_lte(a, b):
            return tf.sigmoid(5*(b/a - 1), name='smooth_lte' )
            #return tf.where(a <= b, tf.ones(tf.shape(a)), tf.zeros(tf.shape(a)))
        def smooth_gte(a, b):
            return smooth_lte(-b, -a)
        
        
    with tf.name_scope("simulator"):
        with tf.name_scope("slicing"):
            buy_price       = trader[:,:, RESPONSE_BUY_PRICE]
            buy_amount      = trader[:,:, RESPONSE_BUY_AMOUNT]
            sell_low_price  = trader[:,:, RESPONSE_SELL_LOW]
            sell_high_price = trader[:,:, RESPONSE_SELL_HIGH]
            current_money   = state.money[:, MONEY_TODAY]
            current_stocks  = state.stocks
            eod_stock_prices = next_day_data[:, :, FEATURE_CLOSE]

        # Executes SELL transactions if prices reaches BELOW a threshold (minimize losses)            
        # (If the day opens below the threshold for LOW SELL, use the open price)
        with tf.name_scope("sell_low"):
            will_sell_low         = smooth_lte(next_day_data[:, :, FEATURE_LOW],  sell_low_price)
            will_sell_low         = tf.where(current_stocks <= 0, tf.zeros(tf.shape(will_sell_low)), will_sell_low)
            actual_sell_low_price = sell_low_price # FIXMEtf.min(next_day_data[:, :, FEATURE_OPEN], sell_low_price)
            money_earned_sell_low = tf.reduce_sum(will_sell_low * (actual_sell_low_price * current_stocks), axis=1)
            money_spent_sell_low  = tf.reduce_sum(will_sell_low * SELL_TRANSACTION_COST, axis=1)
            current_stocks        = (1 - will_sell_low) * current_stocks
            kernel                 = tf.identity(will_sell_low, 'kernel')

        # Executes SELL transactions if prices reaches ABOVE a threshold (Maximize gains)
        with tf.name_scope("sell_high"):
            will_sell_high         = smooth_gte(next_day_data[:, :, FEATURE_HIGH], sell_high_price)
            will_sell_high         = tf.where(current_stocks <= 0, tf.zeros(tf.shape(will_sell_high)), will_sell_high)
            money_earned_sell_high = tf.reduce_sum(will_sell_high * (sell_high_price * current_stocks), axis=1)
            money_spent_sell_high  = tf.reduce_sum(will_sell_high * SELL_TRANSACTION_COST, axis=1)
            current_stocks         = (1 - will_sell_high) * current_stocks
            kernel                 = tf.identity(will_sell_high, 'kernel')

        # Executes BUY transactions if prices reaches above a threshold
        # (Normalize `buy_amount` to be in amount of stocks, instead of fraction of my money)
        with tf.name_scope("buy"):
            buy_amount       = tf.floor(buy_amount * current_money[:, tf.newaxis] / buy_price)
            will_buy         = smooth_lte(next_day_data[:, :, FEATURE_LOW], buy_price)
            will_buy         = tf.where(buy_amount <= 0, tf.zeros(tf.shape(will_buy)), will_buy)
            money_spent_buy  = tf.reduce_sum(will_buy * ((buy_price * buy_amount) + BUY_TRANSACTION_COST), axis=1)
            current_stocks   = current_stocks + will_buy * buy_amount            
            kernel           = tf.identity(will_buy, 'kernel')

        with tf.name_scope("update_money"):
            with tf.name_scope("money_spent"):
                money_spent = money_spent_sell_low + money_spent_sell_high + money_spent_buy
            with tf.name_scope("money_earned"):
                money_earned = money_earned_sell_low + money_earned_sell_high
            
            new_money_first = tf.expand_dims(current_money + state.money[:, MONEY_TOMORROW] - money_spent, axis = 1)
            new_money_middle = state.money[:, 2:]
            new_money_last = tf.expand_dims(money_earned, axis = 1)
            
            next_money = tf.concat([
                new_money_first,
                new_money_middle,
                new_money_last
            ], axis = 1)
            #print(f"MONEY - IN: {state.money.shape}")#, OUT: {next_money.shape}")
            #print(f"current_money:{current_money.shape}, money_spent:{money_spent.shape}, money_earned_low:{money_earned_low.shape}, money_earned_high:{money_earned_high.shape}")
            #print(f"new_first:{new_money_first.shape}, new_middle:{new_money_middle.shape}, new_last:{new_money_last.shape}, money_earned_high:{money_earned_high.shape}")
            #print(f"MONEY - OUT: {next_money.shape}")
            
        return TradingState(
            stocks = current_stocks,
            money = next_money,
            account_value = calculate_account_value(next_money, current_stocks, eod_stock_prices),
            inner_state = new_inner_state
        )       
        

In [181]:
def build_env(trainable_variables_node, historic_data, initial_state, num_days, historic_window_size):    
    first_state_node = tf.name_scope(f'state_0')
    
    build_trader, first_inner_state = trader_builder(historic_window_size, trainable_variables_node, first_state_node)
    
    with first_state_node:
        current_state = TradingState(
            stocks = tf.identity(initial_state.stocks, name='stocks'),
            money = tf.identity(initial_state.money, name='money'),
            account_value = tf.identity(initial_state.account_value, name='account_value'),
            inner_state = first_inner_state
        )
    all_states = [current_state]
    
        
    for i in range(num_days):
        #print(f'day_{i}')
        with tf.name_scope(f'day_{i}'):
            with tf.name_scope(f'historic_data'):
                historic_slice = historic_data[:, i:i+historic_window_size, : ,:]
            with tf.name_scope(f'next_day_data'):
                next_day_slice = historic_data[:, i+historic_window_size, : ,:]

            current_state = build_env_step(
                build_trader = build_trader,
                state = current_state,
                historic_data = historic_slice,
                next_day_data = next_day_slice
            )
            
        with tf.name_scope(f'state_{i+1}'):
            current_state = TradingState(
                stocks = tf.identity(current_state.stocks, name='stocks'),
                money = tf.identity(current_state.money, name='money'),
                account_value = tf.identity(current_state.account_value, name='account_value'),
                inner_state = None
            )

        all_states.append(current_state)
    return all_states

In [219]:
def build_graph(warmup_days = 5, evaluated_days = 20, historic_window_size = 5, money_settle_time = 3) -> tf.Graph:
    total_days = evaluated_days + warmup_days + historic_window_size
    
    graph = tf.Graph()
    with graph.as_default():
        trainable_variables_node = tf.name_scope("state")
        with trainable_variables_node:
            iteration = tf.Variable(0, name='iteration', dtype=tf.int32, expected_shape=())
                    
        with tf.name_scope("inputs"):
            stock_history = tf.placeholder(tf.float32, shape=(None, total_days, None, 5), name="stock_history")

            minibatch_size = tf.shape(stock_history)[0]
            num_companies = tf.shape(stock_history)[2]    
            default_initial_stocks = tf.zeros([minibatch_size, num_companies], name = 'default_initial_stocks')
            initial_stocks = tf.placeholder_with_default(default_initial_stocks, shape = (None, None), name = 'initial_stocks')

            initial_money_simple = tf.placeholder_with_default(10000., shape = (), name = 'initial_money_simple')
            default_initial_money = tf.concat(
                [
                    initial_money_simple * tf.ones([minibatch_size, 1]),
                    tf.zeros([minibatch_size, money_settle_time-1])
                ], 
                axis = 1,
                name = 'default_initial_money')
            initial_money = tf.placeholder_with_default(default_initial_money, shape = (None, money_settle_time), name = 'initial_money')

            initial_stock_prices = stock_history[:, historic_window_size - 1, :, FEATURE_CLOSE]
            
            initial_state = TradingState(
                stocks = initial_stocks,
                money = initial_money,
                account_value = calculate_account_value(initial_money, initial_stocks, initial_stock_prices),
                inner_state = None
            )

        with tf.name_scope("trading"):
            all_states = build_env(trainable_variables_node, stock_history, initial_state, warmup_days + evaluated_days, historic_window_size)

        with tf.name_scope("evaluation"):
            with tf.name_scope("account_values"):
                account_values = tf.stack([
                    all_states[i].account_value
                    for i in range(warmup_days, warmup_days + evaluated_days + 1)
                ], axis = 1, name = 'values')
            with tf.name_scope("daily_variation"):
                daily_variation = tf.div(account_values[:, 1:], account_values[:, 0:-1], name = "values")
            with tf.name_scope("normalized_daily_variation"):
                normalized_daily_variation = tf.multiply(100., tf.log(daily_variation), name = "values")           
                tf.summary.histogram('normalized_daily_variation', normalized_daily_variation)
                
                mean_normalized_daily_variation = tf.reduce_mean(normalized_daily_variation, name = "mean")
                tf.summary.scalar('mean_normalized_daily_variation', mean_normalized_daily_variation)
                
    return graph    

In [ ]:
graph = build_graph()
show_graph(graph)

In [217]:
DATASET_DIR = "dataset/dataset-2017-10-11"
STOCK_DIR = f"{DATASET_DIR}/Stocks"
ETF_DIR = f"{DATASET_DIR}/ETFs"

sp500 = pd.read_csv('dataset/s&p500.tsv', sep='\t')['Ticker symbol']

def concat_datasets(datasets):
    return pd.concat(
        df.assign(Symbol = symbol)
        for symbol, df in datasets.items()
    )

def dataset_to_timeseries(df, days):
    """
    Concatenates `days` sequential values to create a larger feature array.
    """
    parallel_series = [
        df[d:len(df)+1-days+d]
        for d in reversed(range(days))
    ]
    
    cols = ['Date']
    data = [parallel_series[0].Date]
    for i in range(len(parallel_series)):
        s = parallel_series[i]
        for col in ['Open', 'Close', 'High', 'Low', 'Volume']:
            cols.append(f'{col}.{i}')
            data.append(s[col])
            
    return pd.DataFrame(list(zip(*data)), columns=cols)

def split_dataset_by_date(dataset, test_size = 0.2):
    trading_dates = sorted(set(dataset.Date))
    train_dates, test_dates = train_test_split(trading_dates, test_size=test_size)
    train_dataset = dataset[dataset.Date.isin(set(train_dates))].sample(frac=1).reset_index(drop=True)
    test_dataset = dataset[dataset.Date.isin(set(test_dates))].sample(frac=1).reset_index(drop=True)
    return (train_dataset, test_dataset)

def extract_all_data(symbols, feature_days=30, min_date='1990-01-01'):
    symbols = set(symbols)
    failed_csvs = []
    raw_data = {}
    timeseries_feature_data = {}
    for filename in os.listdir(STOCK_DIR):
        symbol = filename.split('.')[0].upper()
        if not symbol in symbols:
            continue

        raw = pd.read_csv(f"{STOCK_DIR}/{filename}")
        raw_data[symbol] = raw[raw.Date >= min_date]
        timeseries_feature_data[symbol] = dataset_to_timeseries(raw_data[symbol], feature_days)

        #if len(raw_data) > 5:
            #break
            
    if failed_csvs:
        logging.warning(f'Failed to read {len(failed_csvs)} CSV files: {failed_csvs}')
            
    all_samples = concat_datasets(timeseries_feature_data)
    train_samples, test_samples = split_dataset_by_date(all_samples)
    return train_samples, test_samples

def create_minibatch(dataset, minibatch_size, num_companies):
    minibatch = []
    for i in range(minibatch_size):
        date = random.choice(dataset.Date)
        dataset_at_date = dataset[dataset.Date == date]

        samples = dataset_at_date.sample(n = num_companies, replace = True)
        samples = samples.as_matrix(columns=samples.columns[1:-1])
        samples = samples.reshape([num_companies, -1, 5])
        samples = samples.transpose((1,0,2))
        minibatch.append(samples)
    return np.stack(minibatch)
    
def minibatch_producer(symbols, feature_days=30, min_date='2010-01-01', minibatch_size=10, num_companies=10):
    all_data = extract_all_data(symbols, feature_days=feature_days, min_date=min_date)
    def produce(train=True, minibatch_size=minibatch_size, num_companies=num_companies):
        return create_minibatch(dataset=all_data[0 if train else 1], minibatch_size=minibatch_size, num_companies=num_companies)
    return produce

next_minibatch = minibatch_producer(sp500[:20])

In [313]:
with build_graph().as_default():    
    checkpoint_file = os.path.abspath(f'checkpoint/trader.ckpt')
    tensorboard_file = os.path.abspath(f'tensorboard/{datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")}')
    os.makedirs(os.path.dirname(checkpoint_file), exist_ok=True)
    os.makedirs(os.path.dirname(tensorboard_file), exist_ok=True)
    
    input_stock_history = tf.get_default_graph().get_tensor_by_name('inputs/stock_history:0')
    input_initial_money = tf.get_default_graph().get_tensor_by_name('inputs/initial_money_simple:0')

    iteration = tf.get_default_graph().get_tensor_by_name('state/iteration:0')
    
    buy_coefs = tf.get_default_graph().get_tensor_by_name('trading/state/buy_coefs:0')
    sell_coefs = tf.get_default_graph().get_tensor_by_name('trading/state/sell_coefs:0')

    
    evaluation_normalized_profit = tf.get_default_graph().get_tensor_by_name('evaluation/normalized_daily_variation/values:0')
    evaluation_normalized_profit_mean = tf.get_default_graph().get_tensor_by_name('evaluation/normalized_daily_variation/mean:0')
    all_summaries = tf.summary.merge_all()                 
    
    optimizer = tf.train.MomentumOptimizer(learning_rate=0.001, momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(-evaluation_normalized_profit_mean, global_step = iteration)
    
    saver = tf.train.Saver()
    with tf.Session() as session:        
        with tf.summary.FileWriter(tensorboard_file, tf.get_default_graph()) as tensorboard_writer:    
            tf.global_variables_initializer().run()
            try:
                #saver.restore(session, checkpoint_file)
                pass
            except:
                logging.warning(f'Failed to restore training state from {checkpoint_file}')
                raise
                
            last_checkout = time() - 1
            while True:
                now = time()
                if (now - last_checkout) >= 1:
                    last_checkout = now
                    saver.save(session, checkpoint_file)
                    
                    #print([iteration, evaluation_normalized_profit, all_summaries])
                    test_params = {
                        tf.get_default_graph().get_tensor_by_name('inputs/stock_history:0'): next_minibatch(train=False, minibatch_size=100, num_companies=10),
                        tf.get_default_graph().get_tensor_by_name('inputs/initial_money_simple:0'): 50000
                    }
                    result = session.run([iteration, evaluation_normalized_profit, all_summaries], feed_dict=test_params)
                    i = result[0]
                    normalized_evaluation_values = result[1]
                    summaries = result[2]
                                
                    print(f'Epoch [{i}] -- trader_evaluation: {normalized_evaluation_values.mean():.2f}±{normalized_evaluation_values.std():.2f} %/day')
                    tensorboard_writer.add_summary(summaries, i)

                train_params = {
                    tf.get_default_graph().get_tensor_by_name('inputs/stock_history:0'): next_minibatch(train=True, minibatch_size=100, num_companies=10),
                    tf.get_default_graph().get_tensor_by_name('inputs/initial_money_simple:0'): 50000
                }
                session.run(training_op, feed_dict=train_params)

Epoch [0] -- trader_evaluation: -0.19±0.42 %/day
Epoch [1] -- trader_evaluation: -0.18±0.41 %/day
Epoch [2] -- trader_evaluation: -0.20±0.39 %/day
Epoch [3] -- trader_evaluation: -0.18±0.40 %/day
Epoch [4] -- trader_evaluation: -0.18±0.35 %/day
Epoch [5] -- trader_evaluation: -0.17±0.33 %/day
Epoch [6] -- trader_evaluation: -0.15±0.39 %/day
Epoch [7] -- trader_evaluation: -0.15±0.41 %/day
Epoch [8] -- trader_evaluation: -0.13±0.35 %/day
Epoch [9] -- trader_evaluation: -0.14±0.40 %/day
Epoch [10] -- trader_evaluation: -0.12±0.42 %/day
Epoch [11] -- trader_evaluation: -0.08±0.42 %/day
Epoch [12] -- trader_evaluation: -0.07±0.42 %/day
Epoch [13] -- trader_evaluation: -0.07±0.40 %/day
Epoch [14] -- trader_evaluation: -0.05±0.42 %/day
Epoch [15] -- trader_evaluation: -0.03±0.40 %/day
Epoch [16] -- trader_evaluation: -0.02±0.40 %/day
Epoch [17] -- trader_evaluation: -0.02±0.41 %/day
Epoch [18] -- trader_evaluation: 0.02±0.39 %/day
Epoch [19] -- trader_evaluation: 0.04±0.39 %/day
Epoch [20] -

Epoch [166] -- trader_evaluation: 1.69±0.80 %/day
Epoch [167] -- trader_evaluation: 1.67±0.74 %/day
Epoch [168] -- trader_evaluation: 1.71±0.78 %/day
Epoch [169] -- trader_evaluation: 1.67±0.78 %/day
Epoch [170] -- trader_evaluation: 1.63±0.75 %/day
Epoch [171] -- trader_evaluation: 1.68±1.11 %/day
Epoch [172] -- trader_evaluation: 1.77±0.86 %/day
Epoch [173] -- trader_evaluation: 1.64±0.79 %/day
Epoch [174] -- trader_evaluation: 1.66±0.78 %/day
Epoch [175] -- trader_evaluation: 1.73±0.83 %/day
Epoch [176] -- trader_evaluation: 1.68±0.72 %/day
Epoch [177] -- trader_evaluation: 1.82±0.81 %/day
Epoch [178] -- trader_evaluation: 1.71±0.78 %/day
Epoch [179] -- trader_evaluation: 1.77±0.84 %/day
Epoch [180] -- trader_evaluation: 1.88±0.85 %/day
Epoch [181] -- trader_evaluation: 1.82±0.81 %/day
Epoch [182] -- trader_evaluation: 1.72±0.77 %/day
Epoch [183] -- trader_evaluation: 1.79±0.80 %/day
Epoch [184] -- trader_evaluation: 1.74±0.75 %/day
Epoch [185] -- trader_evaluation: 1.80±0.76 %/day


Epoch [330] -- trader_evaluation: 2.54±0.95 %/day
Epoch [331] -- trader_evaluation: 2.53±1.02 %/day
Epoch [332] -- trader_evaluation: 2.49±0.91 %/day
Epoch [333] -- trader_evaluation: 2.45±0.90 %/day
Epoch [334] -- trader_evaluation: 2.50±0.98 %/day
Epoch [335] -- trader_evaluation: 2.52±1.00 %/day
Epoch [336] -- trader_evaluation: 2.43±0.86 %/day
Epoch [337] -- trader_evaluation: 2.40±0.85 %/day
Epoch [338] -- trader_evaluation: 2.54±0.90 %/day
Epoch [339] -- trader_evaluation: 2.52±0.99 %/day
Epoch [340] -- trader_evaluation: 2.50±0.88 %/day
Epoch [341] -- trader_evaluation: 2.58±1.01 %/day
Epoch [342] -- trader_evaluation: 2.57±0.92 %/day
Epoch [343] -- trader_evaluation: 2.41±0.85 %/day
Epoch [344] -- trader_evaluation: 2.49±0.91 %/day
Epoch [345] -- trader_evaluation: 2.52±0.97 %/day
Epoch [346] -- trader_evaluation: 2.52±0.95 %/day
Epoch [347] -- trader_evaluation: 2.52±0.97 %/day
Epoch [348] -- trader_evaluation: 2.62±1.06 %/day
Epoch [349] -- trader_evaluation: 2.50±0.91 %/day


Epoch [494] -- trader_evaluation: 3.00±1.04 %/day
Epoch [495] -- trader_evaluation: 2.87±0.89 %/day
Epoch [496] -- trader_evaluation: 2.93±0.93 %/day
Epoch [497] -- trader_evaluation: 2.93±1.05 %/day
Epoch [498] -- trader_evaluation: 2.99±1.09 %/day
Epoch [499] -- trader_evaluation: 2.89±1.03 %/day
Epoch [500] -- trader_evaluation: 2.92±0.91 %/day
Epoch [501] -- trader_evaluation: 3.08±2.22 %/day
Epoch [502] -- trader_evaluation: 3.05±1.06 %/day
Epoch [503] -- trader_evaluation: 2.99±1.00 %/day
Epoch [504] -- trader_evaluation: 3.02±2.34 %/day
Epoch [505] -- trader_evaluation: 2.94±1.07 %/day
Epoch [506] -- trader_evaluation: 3.04±1.61 %/day
Epoch [507] -- trader_evaluation: 3.03±1.00 %/day
Epoch [508] -- trader_evaluation: 3.03±1.00 %/day
Epoch [509] -- trader_evaluation: 2.95±1.60 %/day
Epoch [510] -- trader_evaluation: 3.13±2.39 %/day
Epoch [511] -- trader_evaluation: 2.91±0.92 %/day
Epoch [512] -- trader_evaluation: 3.04±0.95 %/day
Epoch [513] -- trader_evaluation: 3.00±1.28 %/day


Epoch [658] -- trader_evaluation: 3.41±3.47 %/day
Epoch [659] -- trader_evaluation: 3.31±3.96 %/day
Epoch [660] -- trader_evaluation: 3.21±0.96 %/day
Epoch [661] -- trader_evaluation: 3.33±2.45 %/day
Epoch [662] -- trader_evaluation: 3.22±0.94 %/day
Epoch [663] -- trader_evaluation: 3.15±0.92 %/day
Epoch [664] -- trader_evaluation: 3.30±2.97 %/day
Epoch [665] -- trader_evaluation: 3.27±1.05 %/day
Epoch [666] -- trader_evaluation: 3.24±1.02 %/day
Epoch [667] -- trader_evaluation: 3.33±0.96 %/day
Epoch [668] -- trader_evaluation: 3.36±1.03 %/day
Epoch [669] -- trader_evaluation: 3.36±3.07 %/day
Epoch [670] -- trader_evaluation: 3.21±0.98 %/day
Epoch [671] -- trader_evaluation: 3.68±7.73 %/day
Epoch [672] -- trader_evaluation: 3.39±3.53 %/day
Epoch [673] -- trader_evaluation: 3.29±1.02 %/day
Epoch [674] -- trader_evaluation: 3.25±0.88 %/day
Epoch [675] -- trader_evaluation: 3.32±1.03 %/day
Epoch [676] -- trader_evaluation: 3.29±1.00 %/day
Epoch [677] -- trader_evaluation: 3.32±0.94 %/day


Epoch [822] -- trader_evaluation: 3.73±9.10 %/day
Epoch [823] -- trader_evaluation: 3.69±10.85 %/day
Epoch [824] -- trader_evaluation: 3.56±1.05 %/day
Epoch [825] -- trader_evaluation: 3.71±14.17 %/day
Epoch [826] -- trader_evaluation: 3.49±1.00 %/day
Epoch [827] -- trader_evaluation: 3.46±1.04 %/day
Epoch [828] -- trader_evaluation: 3.42±0.91 %/day
Epoch [829] -- trader_evaluation: 3.46±0.99 %/day
Epoch [830] -- trader_evaluation: 3.43±0.99 %/day
Epoch [831] -- trader_evaluation: 3.56±1.04 %/day
Epoch [832] -- trader_evaluation: 3.82±14.62 %/day
Epoch [833] -- trader_evaluation: 3.48±0.87 %/day
Epoch [834] -- trader_evaluation: 3.51±1.00 %/day
Epoch [835] -- trader_evaluation: 3.47±1.05 %/day
Epoch [836] -- trader_evaluation: 3.50±0.93 %/day
Epoch [837] -- trader_evaluation: 3.48±0.96 %/day
Epoch [838] -- trader_evaluation: 3.44±0.93 %/day
Epoch [839] -- trader_evaluation: 3.68±9.31 %/day
Epoch [840] -- trader_evaluation: 3.50±1.01 %/day
Epoch [841] -- trader_evaluation: 3.83±10.04 %/

Epoch [985] -- trader_evaluation: 3.52±0.97 %/day
Epoch [986] -- trader_evaluation: 3.49±0.90 %/day
Epoch [987] -- trader_evaluation: 3.48±0.89 %/day
Epoch [988] -- trader_evaluation: 3.57±0.98 %/day
Epoch [989] -- trader_evaluation: 3.59±1.03 %/day
Epoch [990] -- trader_evaluation: 3.54±0.99 %/day
Epoch [991] -- trader_evaluation: 3.46±0.93 %/day
Epoch [992] -- trader_evaluation: 3.50±0.90 %/day
Epoch [993] -- trader_evaluation: 3.59±0.98 %/day
Epoch [994] -- trader_evaluation: 3.89±17.05 %/day
Epoch [995] -- trader_evaluation: 3.53±0.93 %/day
Epoch [996] -- trader_evaluation: 3.54±0.94 %/day
Epoch [997] -- trader_evaluation: 3.87±17.31 %/day
Epoch [998] -- trader_evaluation: 3.50±0.90 %/day
Epoch [999] -- trader_evaluation: 3.65±1.01 %/day
Epoch [1000] -- trader_evaluation: 4.06±17.95 %/day
Epoch [1001] -- trader_evaluation: 3.55±0.97 %/day
Epoch [1002] -- trader_evaluation: 3.56±0.99 %/day
Epoch [1003] -- trader_evaluation: 4.01±18.23 %/day
Epoch [1004] -- trader_evaluation: 3.50±0.

Epoch [1145] -- trader_evaluation: 3.62±0.96 %/day
Epoch [1146] -- trader_evaluation: 5.42±43.56 %/day
Epoch [1147] -- trader_evaluation: 4.77±35.45 %/day
Epoch [1148] -- trader_evaluation: 3.67±0.97 %/day
Epoch [1149] -- trader_evaluation: 3.60±0.91 %/day
Epoch [1150] -- trader_evaluation: 3.65±0.90 %/day
Epoch [1151] -- trader_evaluation: 4.82±35.05 %/day
Epoch [1152] -- trader_evaluation: 4.21±25.33 %/day
Epoch [1153] -- trader_evaluation: 3.62±0.95 %/day


KeyboardInterrupt: 

In [309]:
test_space = np.linspace(0.99, +1.01, 11)
with tf.Graph().as_default():    
    with tf.Session() as session: 
        def smooth_lte(a, b):
            #return tf.where(a <= b, tf.ones(tf.shape(a)), tf.zeros(tf.shape(a)))
            return tf.sigmoid( 500*(b/a - 1), name='smooth_lte' )
            #return tf.where(a <= b, tf.ones(tf.shape(a)), tf.zeros(tf.shape(a)))
        def smooth_gte(a, b):
            return smooth_lte(-b, -a)
        
        def stacked_lte(a):
            return tf.stack([smooth_lte(a, tf.ones(tf.shape(a))), a])
        def stacked_gte(a):
            return tf.stack([smooth_lte(tf.ones(tf.shape(a)), a), a])
        
        np.set_printoptions(suppress=True, precision=3)
        print(session.run(stacked_lte(test_space)))
        print(session.run(stacked_gte(test_space)))
        

[[0.994 0.983 0.953 0.882 0.731 0.5   0.269 0.12  0.048 0.019 0.007]
 [0.99  0.992 0.994 0.996 0.998 1.    1.002 1.004 1.006 1.008 1.01 ]]
[[0.007 0.018 0.047 0.119 0.269 0.5   0.731 0.881 0.953 0.982 0.993]
 [0.99  0.992 0.994 0.996 0.998 1.    1.002 1.004 1.006 1.008 1.01 ]]
